# Chapter 11: Training Deep Neural Networks

Melatih Neural Network yang sangat dalam (DNN) memiliki tantangan tersendiri dibandingkan dengan network dangkal. Bab ini membahas berbagai teknik untuk mengatasi masalah utama dalam Deep Learning agar pelatihan menjadi lebih cepat dan model berkinerja lebih baik.

## Masalah Utama dalam Pelatihan DNN:
1. **Vanishing/Exploding Gradients**: Gradien menjadi terlalu kecil atau terlalu besar saat backpropagation.
2. **Kurangnya Data Berlabel**: Sulit mendapatkan data yang sudah dianotasi untuk tugas kompleks.
3. **Pelatihan Lambat**: DNN besar membutuhkan waktu lama untuk konvergen.
4. **Overfitting**: Model dengan jutaan parameter sangat rentan menghafal data latih.

## 1. Mengatasi Vanishing & Exploding Gradients

Salah satu solusi utama adalah menggunakan teknik inisialisasi bobot yang tepat sesuai dengan fungsi aktivasi yang digunakan.

### Inisialisasi He
Digunakan terutama untuk fungsi aktivasi **ReLU** dan variannya (Leaky ReLU, ELU).

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Contoh penggunaan He Initialization di Keras
layer = keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

## 2. Fungsi Aktivasi Non-Saturasi

Fungsi Sigmoid cenderung mengalami saturasi (gradien mendekati 0) pada nilai input yang ekstrem. Alternatif yang lebih baik meliputi:
- **Leaky ReLU**: Memiliki slope kecil pada nilai negatif untuk mencegah 'dead neurons'.
- **ELU (Exponential Linear Unit)**: Lebih halus di sekitar nol dan seringkali konvergen lebih cepat.
- **SELU**: Dapat menormalkan network secara mandiri (self-normalization) jika arsitekturnya tepat.

In [ ]:
# Menggunakan Leaky ReLU
model = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2)
])

# Menggunakan SELU untuk Self-Normalization
layer_selu = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

## 3. Batch Normalization

Teknik ini menormalkan input di setiap lapisan selama pelatihan, yang secara drastis mengurangi masalah gradien dan memungkinkan penggunaan learning rate yang lebih tinggi.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

## 4. Transfer Learning (Reusing Layers)

Daripada melatih dari awal, kita bisa menggunakan lapisan bawah dari model yang sudah terlatih untuk tugas yang mirip.

In [ ]:
# Misalkan kita memuat model A
# model_A = keras.models.load_model("my_model_A.h5")

# Membuat model B berdasarkan model A tanpa lapisan output terakhir
# model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Membekukan (Freezing) lapisan yang digunakan kembali
# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = False

# Jangan lupa untuk compile setelah mengubah status trainable
# model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

## 5. Optimizer Cepat

Beberapa optimizer populer yang lebih cepat dari Stochastic Gradient Descent (SGD) standar:
- **Momentum Optimization**: Mempercepat gradien di arah yang benar (seperti bola menggelinding).
- **Nesterov Accelerated Gradient (NAG)**: Variasi momentum yang lebih cerdas dalam 'melihat ke depan'.
- **RMSProp**: Menyesuaikan learning rate secara adaptif.
- **Adam**: Menggabungkan ide Momentum dan RMSProp.

In [ ]:
# Menggunakan Adam Optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## 6. Regularisasi untuk Mencegah Overfitting

### Dropout
Teknik di mana setiap neuron memiliki probabilitas (misal 20%) untuk 'dimatikan' selama iterasi pelatihan. Ini memaksa network untuk tidak terlalu bergantung pada neuron spesifik.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

## Kesimpulan Praktis
Berdasarkan bab ini, konfigurasi default yang disarankan untuk DNN umum adalah:
- **Inisialisasi**: He Initialization.
- **Aktivasi**: ELU (atau ReLU jika mengutamakan kecepatan).
- **Normalisasi**: Batch Normalization (terutama untuk network dalam).
- **Regularisasi**: Early Stopping + Dropout.
- **Optimizer**: Adam atau Nadam.
- **Learning Rate**: Menggunakan scheduler seperti 1cycle.